[![DOI](https://zenodo.org/badge/940091341.svg)](https://doi.org/10.5281/zenodo.14939868) <a href="https://www.globh2e.org.au/"><img src="https://img.shields.io/badge/ARC:Funding%20number-IC200100023-blue.svg"/></a>

# **Global Safe Operating space (SOS)**
<!-- **<div style="text-align: left; font-size: 19px;"> Global Safe Operating space (SOS)**</div> -->

<div style="text-align: left; font-size: 16px;">Michaël Lejeune<sup>a,b</sup>, Sami Kara<sup>a,b</sup>, Michael Zwicky Hauschild<sup>c,d</sup>, Sareh Sharabifarahni<sup>a</sup>, Rahman Daiyan<sup>b,e</sup></div><br>

<div style="text-align: left; font-size: 13px;"><sup>a</sup>Sustainability in Manufacturing and Life Cycle Engineering Research Group, School of Mechanical and Manufacturing Engineering, the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>b</sup>Australian Research Council Training Centre for the Global Hydrogen Economy (GlobH2e), the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>c</sup>Centre for Absolute Sustainability, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>d</sup>Division for Quantitative Sustainability Assessment (QSA), Department of Environmental and Resource Engineering, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>e</sup>School of Minerals and Energy Engineering, The University of New South Wales, Sydney 2052, Australia</div><br>

<div style="text-align: left; font-size: 13px;"> Code developer and maintainer:<a href="mailto:m.lejeune@unsw.edu.au"> Michaël Lejeune</div></a><br>


# **0. Initialisation**

In [1]:
using CSV,DataFrames,XLSX,Statistics
using LinearAlgebra,SparseArrays
using JLD2
include("Utils/2_01_utils.jl")
include("Utils/general_utils/ssp_utils.jl")


writetable (generic function with 2 methods)

# **1. Global Safe operating Space**

In the [previous notebook](2_00_direct_human_impact.ipynb), we have highlighted that we need to define the global safe operating space vector $\boldsymbol{\Delta X^{PB}}$. This vector is obtained using equation 11 which we recall here:

$$\boldsymbol{\Delta X^{PB} = \left|X^{PB}-X^0\right|} \tag{11} $$

In the event of a transgression, it is also very useful to know in what zone our impact is located. Indeed the planetary boundaries framework has three distinct zones. The first one is defined by the safe operating space (SOS). In this zone, human activities are sustainable. Beyond the safe operating space $1 \leq d_i \leq 1$, we enter a zone of increasing risk denoter by the superscript $^R$ and the danger zone denoted by the $^D$. Hence, it is possible to define equation 12 where the total space up until the danger zone $\boldsymbol{\Delta X^{D}}$ is simply delimited by a known value $X^{D}$ known to be in the danger zone .

$$\boldsymbol{\Delta X^{D} = \left|X^{D}-X^0\right|} \tag{12} $$

Hence more generally we can write 

$$ \boldsymbol{\Delta X^{b} = \left|X^{b}-X^0\right|} \quad \forall b \in \{PB, D\} \tag{12} $$

Below we extract values from Ridchardson et al. 2023 except for the Biosphere integrity which we retrieve from Bachmann et al.2023 and Lade et al. 2020.

In [3]:
𝐗⁰=[0, 278, 3.44, 0.03, 0, 0, 0, 290, 100, 100] # Pre-industrial values
𝐗ᴾᵇ=[1, 350, 2.752, 0.1, 4000, 11, 62, 276, 75, 90] # planetary boudaries
𝐗ᴰ=[1.5, 450, 2.408, 0.250, 6000, 100, 82, 261, 54, 30] # Danger zone boundary
units = ["W/m2","ppm","Ωarag","Aerosol optical depth","km3","TgP","TgN","Dobson units","%","%"]; # units


Now let's define these 2 environmental spaces.

In [4]:
𝚫𝐗ᴾᵇ = abs.(𝐗ᴾᵇ-𝐗⁰);
𝚫𝐗ᴰ = abs.(𝐗ᴰ-𝐗⁰);


let's now represent this in a nice table to be easily

In [5]:
control_var=["Climate change-Energy imbalance"
            "Climate change-CO2 Concentration"
            "Ocean acidification-Carbonate ion concentration"
            "Atmospheric aerosol loading-Aerosol Optical Depth (AOD)"
            "Freshwater use-Global"
            "Biogeochemical flows-P"
            "Biogeochemical flows-N"
            "Stratospheric ozone depletion-Stratospheric O3 concentration"
            "Land-system change-Global"
            "Biosphere Integrity-Change in biosphere integrity"];

df=DataFrame(hcat(control_var,
                𝐗⁰, # Pre-industrial
                𝐗ᴾᵇ, # Boundary values vector 
                𝚫𝐗ᴾᵇ, # Planetary boundary space
                𝐗ᴰ, # Danger zone boundary
                𝚫𝐗ᴰ, # max carrying capacity space
                units),:auto)

rename!(df, Dict(:x1 =>"Control variable",
                :x2 => "Pre-industrial value", 
                :x3 => "Planetary boundary",
                :x4 => "SOS",
                :x5 => "High risk",
                :x6 => "High risk space", 
                :x7 => "Units"))


Row,Control variable,Pre-industrial value,Planetary boundary,SOS,High risk,High risk space,Units
,Any,Any,Any,Any,Any,Any,Any
1,Climate change-Energy imbalance,0.0,1.0,1.0,1.5,1.5,W/m2
2,Climate change-CO2 Concentration,278.0,350.0,72.0,450.0,172.0,ppm
3,Ocean acidification-Carbonate ion concentration,3.44,2.752,0.688,2.408,1.032,Ωarag
4,Atmospheric aerosol loading-Aerosol Optical Depth (AOD),0.03,0.1,0.07,0.25,0.22,Aerosol optical depth
5,Freshwater use-Global,0.0,4000.0,4000.0,6000.0,6000.0,km3
6,Biogeochemical flows-P,0.0,11.0,11.0,100.0,100.0,TgP
7,Biogeochemical flows-N,0.0,62.0,62.0,82.0,82.0,TgN
8,Stratospheric ozone depletion-Stratospheric O3 concentration,290.0,276.0,14.0,261.0,29.0,Dobson units
9,Land-system change-Global,100.0,75.0,25.0,54.0,46.0,%


In [9]:
writetable(df; workbook=respath*"SOS.xlsx", worksheet="Planetary boundaries")


In [5]:
@save respath*"data_SOS.jld2" 𝚫𝐗ᴾᵇ 𝚫𝐗ᴰ control_var units


# **References**

1. Richardson, K. et al. (2023) ‘Earth beyond six of nine planetary boundaries’, Science Advances, 9(37), p. eadh2458. Available at: https://doi.org/10.1126/sciadv.adh2458.

2. Bachmann, M. et al. (2023) ‘Towards circular plastics within planetary boundaries’, Nature Sustainability, 6(5), pp. 599–610. Available at: https://doi.org/10.1038/s41893-022-01054-9.
